In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, datasets, transforms
from torchsummary import summary
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset 
import numpy as np
import torch.optim as optim

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle

In [4]:
!kaggle datasets download -d mylee2009/shopee-round-2-product-detection-challenge

100% 9.38G/9.38G [03:45<00:00, 34.4MB/s]
100% 9.38G/9.38G [03:46<00:00, 44.5MB/s]


In [5]:
!unzip "/content/shopee-round-2-product-detection-challenge.zip" -d "/content/shopee"

Streaming output truncated to the last 5000 lines.
  inflating: /content/shopee/train/train/40/238399dc6718dfb872962b649c5b8b83.jpg  
  inflating: /content/shopee/train/train/40/23a2c307f8a05a1e21eb32f56e101ed9.jpg  
  inflating: /content/shopee/train/train/40/23ad693e18e097185d1244374ba769cc.jpg  
  inflating: /content/shopee/train/train/40/23bd6cca1e919689206f92388b44d325.jpg  
  inflating: /content/shopee/train/train/40/23c7beac3cdafe529351114903c59fce.jpg  
  inflating: /content/shopee/train/train/40/23e8212b6432efd9425f3892f4027352.jpg  
  inflating: /content/shopee/train/train/40/23ff069da9e694541c35444ebc7f21a7.jpg  
  inflating: /content/shopee/train/train/40/2418076cb2f41c86fa1e91b385768093.jpg  
  inflating: /content/shopee/train/train/40/2437204a3cfe46b863373820961d0d46.jpg  
  inflating: /content/shopee/train/train/40/24378c3a9bf723b99e05757fedb1ea87.jpg  
  inflating: /content/shopee/train/train/40/243794ebb1e40d23766b818ab8aefb87.jpg  
  inflating: /content/shopee/train/t

In [15]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [6]:
DenseNet_161 = models.densenet161(pretrained = True)
for para in DenseNet_161.parameters():
  para.requires_grad = False

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/checkpoints/densenet161-8d451a50.pth


In [7]:
in_features = DenseNet_161.classifier.in_features 
DenseNet_161.classifier = nn.Linear(in_features, 42)
for name, child in DenseNet_161.features.named_children():
  if name in ["denseblock3", "denseblock4"]:
    for para in child.parameters():
      para.requires_grad = True
for para in DenseNet_161.classifier.parameters():
  para.requires_grad = True

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(DenseNet_161)

21127722

In [17]:
transform = transforms.Compose([transforms.CenterCrop(224),
                                transforms.ToTensor(),
                                transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])
Ds_train = datasets.ImageFolder("/content/shopee/train/train", transform = transform)
Dl_train = DataLoader(Ds_train, batch_size = 64, shuffle = True)

In [11]:
loss = nn.CrossEntropyLoss()
plist = [{"params": filter(lambda x: x.requires_grad, DenseNet_161.features.denseblock3.parameters()), "lr": 0.001},
         {"params": filter(lambda x: x.requires_grad, DenseNet_161.classifier.parameters()), "lr": 0.1},
         {"params": filter(lambda x: x.requires_grad, DenseNet_161.features.denseblock4.parameters()), "lr": 0.01}
         ]
optimizer = optim.SGD(plist, lr = 0.001)

In [12]:
!pip install barbar

In [ ]:
from barbar import Bar
import time
DenseNet_161.to(device)
Loss = []
Acc = []
Epoch = []
num_epoch = 25
for epoch in range(4,num_epoch):
    t = time.time()
    DenseNet_161.train()
    sum_loss = 0
    total = 0
    correct = 0
    print('Epoch: %d/%d' % (epoch + 1, num_epoch))
    for X, y in Bar(Dl_train):
        y.long()
        X = X.to(device)
        y = y.to(device)
        optimizer.zero_grad()
        y_pred = DenseNet_161(X)
        loss = F.cross_entropy(y_pred, y)
        loss.backward()
        optimizer.step()
        total += y.size(0)
        sum_loss += loss.item()*(y.size(0))
        _, predicted = torch.max(y_pred, 1)
        correct += (predicted == y).sum().item()
    Loss.append(sum_loss/total)
    Acc.append(correct/total)
    Epoch.append(epoch)
    if epoch == 0:
      print("Executing time: {} s".format(time.time()-t))
    ggdr_path = "/content/drive/My Drive/Weight_Shopee/Densnet-161_Weight_Epoch" + str(epoch) + ".pth"
    torch.save(DenseNet_161.state_dict(), ggdr_path)
    print("'Loss: {} ----- Accuracy: {} %".format(sum_loss/total, (correct*100)/total))
print("The training process is done")
torch.save(model, model_path)
print("model saved")
torch.save(model.state_dict(), weight_path)

Epoch: 5/25
 49792/105392: [===============>................] - ETA 2412.9s